# Observe other vision practices that have moved to large messaging in Spine

On the 23rd March 2021, three more Vision practices enabled large messaging for GP2GP. We want to look at our Spine data for these practices to identify how their failure rates have changed since moving to large messaging. 

### Scope

Analyse spine data for the following three practices, for transfers both in and out:

Went live with large messaging from: 23.03.21 - 12.16pm 

User ID | Practice Code | Practice Name

#668 | M89019 | Hobs Moat Medical Centre

#33018 | M82010 | Drayton Medical Centre

#857 | H84023 | Essex House Surgery

- Document the technical success rate of their transfers for three months prior to 23.03
- Document a high level breakdown of reasons for failures for this time period.
- Document the technical success rate of their transfers since 23.03. 
- Document a high level breakdown of reasons for this time period

 
### Acceptance Criteria
- We are able to compare the technical success rate for these two practices before and after enabling attachments, and communicate to stakeholders how much their failure rates have changed. 
- We are able to tell the Vision team the reasons for some transfers still failing since moving to Large messaging

In [1]:
from datetime import datetime,timedelta

practice_codes=["M89019","M82010","H84023"]
time_of_switch=datetime(year=2021, month=3, day=23, hour=12, minute=16)
window=timedelta(28) # Number of Days before and after switch to consider changes

start_time=time_of_switch-window
end_time=time_of_switch+window

In [7]:
import pandas as pd
import numpy as np

transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-duplicates-hypothesis/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet",
    "3-2021-transfers.parquet",
    "4-2021-transfers.parquet",
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))
# This is only needed when using transfers-duplicates-hypothesis datasets
transfers_raw = transfers_raw.drop(["sending_supplier", "requesting_supplier"], axis=1)




# Given the findings in PRMT-1742 - many duplicate EHR errors are misclassified, the below reclassifies the relevant data
successful_transfers_bool = transfers_raw['request_completed_ack_codes'].apply(lambda x: True in [(np.isnan(i) or i==15) for i in x])
transfers = transfers_raw.copy()
transfers.loc[successful_transfers_bool, "status"] = "INTEGRATED"

# Pending (Sender Error) Code
pending_sender_error_codes=[6,7,10,24,30,23,14,99]
transfers_with_pending_sender_code_bool=transfers['sender_error_code'].isin(pending_sender_error_codes)
transfers_with_pending_with_error_bool=transfers['status']=='PENDING_WITH_ERROR'
transfers_which_need_pending_to_failure_change_bool=transfers_with_pending_sender_code_bool & transfers_with_pending_with_error_bool
transfers.loc[transfers_which_need_pending_to_failure_change_bool,'status']='FAILED'

# Add integrated Late status
eight_days_in_seconds=8*24*60*60
transfers_after_sla_bool=transfers['sla_duration']>eight_days_in_seconds
transfers_with_integrated_bool=transfers['status']=='INTEGRATED'
transfers_integrated_late_bool=transfers_after_sla_bool & transfers_with_integrated_bool
transfers.loc[transfers_integrated_late_bool,'status']='INTEGRATED LATE'


# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

asid_lookup_file = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Mar-2021.csv.gz"
asid_lookup = pd.read_csv(asid_lookup_file)
lookup = asid_lookup[["ASID", "MName", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid', 'NACS': 'requesting_ods_code',"OrgName":'requesting_practice'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code',"OrgName":'sending_practice'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

### Extract the data relevant to the practices in question

In [20]:
requesting_transfers_bool=transfers["requesting_ods_code"].isin(practice_codes)
requesting_transfers=transfers.loc[requesting_transfers_bool].copy()
requesting_transfers['ods_code']=requesting_transfers['requesting_ods_code']
requesting_transfers['transfer type']='Requesting'
requesting_transfers['practice']=requesting_transfers['requesting_practice']

sending_transfers_bool=transfers["sending_ods_code"].isin(practice_codes)
sending_transfers=transfers.loc[sending_transfers_bool].copy()
sending_transfers['ods_code']=sending_transfers['sending_ods_code']
sending_transfers['transfer type']='Sending'
sending_transfers['practice']=sending_transfers['sending_practice']

relevant_transfers=pd.concat([requesting_transfers,sending_transfers],axis=0)

In [21]:
relevant_transfers['Large Messaging Available']=relevant_transfers['date_requested']>time_of_switch
relevant_transfers_in_time_window_bool=(relevant_transfers['date_requested']>=start_time) & (relevant_transfers['date_requested']<=end_time)
relevant_transfers=relevant_transfers.loc[relevant_transfers_in_time_window_bool]

## What is the change in outcomes (status) as a result?

In [24]:
outcomes_table=pd.pivot_table(relevant_transfers,index=['Large Messaging Available'],columns='status',values='conversation_id',aggfunc='count').fillna(0)
outcomes_table_percentage_chance=(100*outcomes_table.div(outcomes_table.sum(axis=1),axis=0)).round(2)
outcomes_table_percentage_chance

status,FAILED,INTEGRATED,INTEGRATED LATE,PENDING,PENDING_WITH_ERROR
Large Messaging Available,,,,,
False,43.78,46.89,1.04,7.88,0.41
True,2.66,82.45,0.53,13.83,0.53


In [22]:
outcomes_table=pd.pivot_table(relevant_transfers,index=['transfer type','Large Messaging Available'],columns='status',values='conversation_id',aggfunc='count').fillna(0)
outcomes_table_percentage_chance=(100*outcomes_table.div(outcomes_table.sum(axis=1),axis=0)).round(2)
outcomes_table_percentage_chance

status                                   FAILED  INTEGRATED  INTEGRATED LATE  \
transfer type Large Messaging Available                                        
Requesting    False                       52.67       39.15             0.00   
              True                         4.21       82.63             0.00   
Sending       False                       31.34       57.71             2.49   
              True                         1.08       82.26             1.08   

status                                   PENDING  PENDING_WITH_ERROR  
transfer type Large Messaging Available                               
Requesting    False                         7.47                0.71  
              True                         12.11                1.05  
Sending       False                         8.46                0.00  
              True                         15.59                0.00

In [23]:
outcomes_table=pd.pivot_table(relevant_transfers,index=["practice",'transfer type','Large Messaging Available'],columns='status',values='conversation_id',aggfunc='count').fillna(0)
outcomes_table_percentage_chance=(100*outcomes_table.div(outcomes_table.sum(axis=1),axis=0)).round(2)
outcomes_table_percentage_chance

status                                                                   FAILED  \
practice                        transfer type Large Messaging Available           
FLOOD (ESSEX HOUSE)             Requesting    False                       47.66   
                                              True                         3.45   
                                Sending       False                       11.76   
                                              True                         1.14   
HOBS MOAT MEDICAL CENTRE        Requesting    False                       52.38   
                                              True                         4.17   
                                Sending       False                       53.23   
                                              True                         2.38   
MARKET DRAYTON MEDICAL PRACTICE Requesting    False                       58.89   
                                              True                         6.00   
                                Sending       False                       30.99   
                                              True                         0.00   

status                                                                   INTEGRATED  \
practice                        transfer type Large Messaging Available               
FLOOD (ESSEX HOUSE)             Requesting    False                           45.79   
                                              True                            82.76   
                                Sending       False                           79.41   
                                              True                            78.41   
HOBS MOAT MEDICAL CENTRE        Requesting    False                           32.14   
                                              True                            87.50   
                                Sending       False                           33.87   
                                              True                            78.57   
MARKET DRAYTON MEDICAL PRACTICE Requesting    False                           37.78   
                                              True                            80.00   
                                Sending       False                           57.75   
                                              True                            91.07   

status                                                                   INTEGRATED LATE  \
practice                        transfer type Large Messaging Available                    
FLOOD (ESSEX HOUSE)             Requesting    False                                 0.00   
                                              True                                  0.00   
                                Sending       False                                 1.47   
                                              True                                  1.14   
HOBS MOAT MEDICAL CENTRE        Requesting    False                                 0.00   
                                              True                                  0.00   
                                Sending       False                                 0.00   
                                              True                                  0.00   
MARKET DRAYTON MEDICAL PRACTICE Requesting    False                                 0.00   
                                              True                                  0.00   
                                Sending       False                                 5.63   
                                              True                                  1.79   

status                                                                   PENDING  \
practice                        transfer type Large Messaging Available            
FLOOD (ESSEX HOUSE)             Requesting    False                         6.54   
                                              True                         12